## Create final plot showing remaining need, based on ndc children served compared to estimated children <4 in TennCare.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
#import NDC deliveries csv
county_stats = pd.read_csv('../files/county_stats.csv')
county_stats.head()

,County,Total Population,Population Age 0-4,Population Age 5-9,Population Age 0-9,# Households,# Live Births,Est # in Diapers,Est. TennCare,Household Income (Avg),% Title 1 Schools,% Food Stamp,% Poverty Rate 0-17,% Poverty Rate All Ages,Pop in Poverty <17,Pop in Poverty Tot,In Diapers in Poverty,diapers used,Diaper cost/month,$ per county/month for low income
0,Davidson,"667,885","46,539","41,537","88,076","269,078","9,962","24,905","12,453","$60,388",63.42%,14.60%,25.40%,15.40%,169643,102854,6325.870,1518208.80,$80.00,"$442,810.90"
1,Williamson,"205,645","12,451","16,755","29,206","71,043","2,321","5,803","2,901","$112,962",17.74%,3.40%,3.50%,3.80%,7198,7815,203.105,48745.20,$80.00,"$14,217.35"
2,Montgomery,"189,709","16,476","14,404","30,880","67,090","3,512","8,780","4,390","$57,541",61.70%,13.20%,16.60%,12.00%,31492,22765,1457.480,349795.20,$80.00,"$102,023.60"
3,Maury,"85,767","5,730","6,077","11,807","32,889","1,184","2,960","1,480","$57,170",64.20%,16.40%,15.20%,10.80%,13037,9263,449.920,107980.80,$80.00,"$31,494.40"
4,Hickman,"24,251","1,291","1,553","2,844","8,855",257,643,321,"$43,596",88.89%,22.70%,24.20%,16.30%,5869,3953,155.606,37345.44,$80.00,"$10,892.42"


In [3]:
#only keep needed columns
county_stats = county_stats[['County',
                             'Population Age 0-4',
                             'Est # in Diapers',
                             'Est. TennCare',
                             'In Diapers in Poverty',
                             'diapers used'
                            ]].copy()
county_stats.tail()

,County,Population Age 0-4,Est # in Diapers,Est. TennCare,In Diapers in Poverty,diapers used
6,Putnam,"4,298","2,175","1,088",406.7250,97614.00
7,Rutherford,"19,515","10,365","5,183",1285.2600,308462.40
8,Cheatham,"2,237","1,125",563,151.8750,36450.00
9,Wilson,"7,549","4,128",2064,408.6225,98069.40
10,Total,"110,312","57,780","28,890",11048.4400,2651625.24


In [4]:
#rename columns
county_stats = county_stats.rename(columns = {'County':'county',
                                              'Population Age 0-4':'pop_under_4',
                                              'Est # in Diapers':'est_in_diapers',
                                              'Est. TennCare':'est_in_tenncare',
                                              'In Diapers in Poverty':'diapers_and_poverty',
                                              'diapers used':'diapers_used'
                                             })
county_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   county               11 non-null     object 
 1   pop_under_4          11 non-null     object 
 2   est_in_diapers       11 non-null     object 
 3   est_in_tenncare      11 non-null     object 
 4   diapers_and_poverty  11 non-null     float64
 5   diapers_used         11 non-null     float64
dtypes: float64(2), object(4)
memory usage: 656.0+ bytes


In [5]:
#drop unneeded rows
county_stats = county_stats.drop(county_stats.index[[1,2,3,4,5,6,7,8,9]]).reset_index()
county_stats = county_stats.drop(columns=['index'])
county_stats

,county,pop_under_4,est_in_diapers,est_in_tenncare,diapers_and_poverty,diapers_used
0,Davidson,"46,539","24,905","12,453",6325.87,1518208.80
1,Total,"110,312","57,780","28,890",11048.44,2651625.24


In [6]:
#drop some more columns
county_stats = county_stats.drop(columns=['est_in_tenncare', 'diapers_and_poverty', 'diapers_used'])
#convert columns to integers
county_stats['pop_under_4'] = county_stats['pop_under_4'].str.replace(',', '').astype(float)
county_stats['est_in_diapers'] = county_stats['est_in_diapers'].str.replace(',', '').astype(float)
#add some columns (16.5% from https://www.sycamoreinstitutetn.org/census-tn-economy-2018/)
county_stats['diaper_pov'] = county_stats['est_in_diapers']*.165
county_stats

,county,pop_under_4,est_in_diapers,diaper_pov
0,Davidson,46539.0,24905.0,4109.325
1,Total,110312.0,57780.0,9533.700


In [7]:
#import NDC deliveries csv
total_by_county = pd.read_csv('../files/total_by_county.csv')
total_by_county

,County,Total
0,"Davidson, TN",1817313
1,"Marshall, TN",2025
2,"Maury, TN",1475
3,"Washington, TN",3750
4,"Williamson, TN",6050
5,"Wilson, TN",10550
6,Total,1841163


In [8]:
#strip TN from counties and rename
total_by_county['County'] = total_by_county['County'].str.split(',').str[0]
total_by_county = total_by_county.rename(columns = {'County':'county',
                                              'Total':'total'
                                             })
total_by_county.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   county  7 non-null      object
 1   total   7 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 240.0+ bytes


In [9]:
#drop unneeded rows
#df.drop(df.index[[2,3]])
total_by_county=total_by_county.drop(total_by_county.index[[1,2,3,4,5]]).reset_index()
total_by_county = total_by_county.drop(columns=['index'])
total_by_county

,county,total
0,Davidson,1817313
1,Total,1841163


In [10]:
#merge datasets
diaper_need = pd.merge(county_stats, total_by_county, on = 'county', how = 'inner')
diaper_need

,county,pop_under_4,est_in_diapers,diaper_pov,total
0,Davidson,46539.0,24905.0,4109.325,1817313
1,Total,110312.0,57780.0,9533.700,1841163
